In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Configs

In [3]:
LABEL2IDX = {
    'O': 0,
    'B-DAT': 1,
    'B-PER': 2,
    'B-ORG': 3,
    'B-LOC': 4,
    'B-EVE': 5,
    'I-DAT': 6,
    'I-PER': 7,
    'I-ORG': 8,
    'I-LOC': 9,
    'I-EVE': 10
}

IDX2LABEL = {i: k for k, i in LABEL2IDX.items()}

CLS = [101]
SEP = [102]
VALUE_TOKEN = [0]
MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
EPOCHS = 4
NUM_CLASS = 11
LEARNING_RATE = 5e-5

In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [6]:
from datasets import concatenate_datasets, load_dataset
from transformers import AutoTokenizer
import ast
# model_name = 'sbunlp/fabert'
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
train= "/content/drive/MyDrive/Colab Notebooks/NER-datasets/24-EVE/train.csv"
eval =  "/content/drive/MyDrive/Colab Notebooks/NER-datasets/24-EVE/eval.csv"
test =  "/content/drive/MyDrive/Colab Notebooks/NER-datasets/24-EVE/test.csv"

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [7]:
def parse_tokens_and_labels(example):
    # Ensure 'tokens' and 'labels' are in the expected format
    if isinstance(example['tokens'], str):
        example['tokens'] = ast.literal_eval(example['tokens'])
    if isinstance(example['labels'], str):
        example['labels'] = ast.literal_eval(example['labels'])
    return example


In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],truncation=True,max_length=512,
 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [9]:
dataset_train = load_dataset('csv', data_files=train)['train']
dataset_eval = load_dataset('csv', data_files=eval)['train']
dataset_test = load_dataset('csv', data_files=test)['train']

dataset_all = concatenate_datasets([dataset_train, dataset_eval, dataset_test])

# Apply the parsing function to the entire dataset
parsed_dataset = dataset_all.map(parse_tokens_and_labels)

# Rename labels to ner_tags
parsed_dataset = parsed_dataset.rename_column("labels", "ner_tags")

# Tokenize and align labels
tokenized_dataset = parsed_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=parsed_dataset.column_names)

# Split the tokenized dataset back into train, eval, and test
dataset_length = len(dataset_train), len(dataset_eval), len(dataset_test)
print(dataset_length)



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

(24000, 3000, 3000)


In [10]:
combined = {
    'train': tokenized_dataset.select(range(dataset_length[0])),
    'eval': tokenized_dataset.select(range(dataset_length[0], dataset_length[0] + dataset_length[1])),
    'test': tokenized_dataset.select(range(dataset_length[0] + dataset_length[1], sum(dataset_length)))
}

In [11]:
combined

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 24000
 }),
 'eval': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 3000
 }),
 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 3000
 })}

In [12]:
# Printing the values and lengths of each column in the first item of tokenized_dataset
print(f"Input IDs: {combined['train'][0]['input_ids']}, length: {len(combined['train'][0]['input_ids'])}")
print(f"Attention Mask: {combined['train'][0]['attention_mask']}, length: {len(combined['train'][0]['attention_mask'])}")
print(f"Labels: {combined['train'][0]['labels']}, length: {len(combined['train'][0]['labels'])}")

Input IDs: [0, 175, 1807, 26468, 222567, 65, 7947, 114095, 412, 138208, 1807, 4226, 14253, 30897, 30042, 96653, 60647, 14253, 30897, 8468, 1287, 376, 22472, 7434, 2523, 65, 8468, 115496, 22472, 7434, 2523, 9123, 22472, 7434, 2523, 122362, 25498, 65, 43847, 14253, 22472, 16563, 178, 2067, 2736, 6, 5, 2], length: 48
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], length: 48
Labels: [-100, 1, 1, 6, -100, 0, 0, -100, 0, 1, 6, 0, 0, -100, 2, 7, -100, 0, -100, 3, 8, -100, 8, 8, 8, 0, 3, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, -100, 0, 0, -100, 0, 0, -100, -100], length: 48


In [13]:
 print(tokenizer.decode(combined['train'][0]['input_ids'][0]))
 print(tokenizer.decode(combined['train'][0]['input_ids'][1]))
 print(tokenizer.decode(combined['train'][0]['input_ids'][-2]))
 print(tokenizer.decode(combined['train'][0]['input_ids'][-1]))

<s>
در
.
</s>


In [14]:
print(tokenizer.convert_ids_to_tokens(combined['train'][0]['input_ids']))


['<s>', '▁در', '▁سال', '▁۱۳', '۸۵', '▁و', '▁ه', 'زمان', '▁با', '▁دومین', '▁سال', '▁ر', '▁رئیس', '▁جمهور', '▁محمود', '▁احمدی', '▁نژاد', '▁رئیس', '▁جمهور', '▁دولت', '▁نه', 'م', '▁جمهوری', '▁اسلامی', '▁ایران', '▁و', '▁دولت', '▁دهم', '▁جمهوری', '▁اسلامی', '▁ایران', '▁نظام', '▁جمهوری', '▁اسلامی', '▁ایران', '▁معاونت', '▁علمی', '▁و', '▁فناوری', '▁رئیس', '▁جمهوری', '▁آغاز', '▁به', '▁کار', '▁کرد', '▁', '.', '</s>']


In [15]:
train_dataset = combined['train']
eval_dataset = combined['eval']
test_dataset = combined['test']
# Print out info of each dataset
print(f"Training set: {train_dataset} samples")
print(f"Validation set: {eval_dataset} samples")
print(f"Test set: {test_dataset} samples")


Training set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 24000
}) samples
Validation set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
}) samples
Test set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
}) samples


In [16]:
from transformers import DataCollatorForTokenClassification
import torch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [17]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

In [18]:
train_sample = train_dataset[0]
print(type(train_sample['input_ids']))

<class 'torch.Tensor'>


In [19]:
# check the paddings with -100s
batch = data_collator([train_dataset[i] for i in range(2)])
batch["labels"]

tensor([[-100,    1,    1,    6, -100,    0,    0, -100,    0,    1,    6,    0,
            0, -100,    2,    7, -100,    0, -100,    3,    8, -100,    8,    8,
            8,    0,    3,    8,    8,    8,    8,    8,    8,    8,    8,    0,
            0,    0,    0,    0, -100,    0,    0, -100,    0,    0, -100, -100],
        [-100,    0,    0,    0,    0, -100,    0,    0,    0, -100,    0,    0,
            0,    0,    0,    3,    8,    0,    0,    0,    0, -100,    0, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [20]:
for i in range(2):
    print(train_dataset[i]["labels"])

tensor([-100,    1,    1,    6, -100,    0,    0, -100,    0,    1,    6,    0,
           0, -100,    2,    7, -100,    0, -100,    3,    8, -100,    8,    8,
           8,    0,    3,    8,    8,    8,    8,    8,    8,    8,    8,    0,
           0,    0,    0,    0, -100,    0,    0, -100,    0,    0, -100, -100])
tensor([-100,    0,    0,    0,    0, -100,    0,    0,    0, -100,    0,    0,
           0,    0,    0,    3,    8,    0,    0,    0,    0, -100,    0, -100,
        -100])


In [ ]:
IDX2LABEL

{0: 'O',
 1: 'B-DAT',
 2: 'B-PER',
 3: 'B-ORG',
 4: 'B-LOC',
 5: 'B-EVE',
 6: 'I-DAT',
 7: 'I-PER',
 8: 'I-ORG',
 9: 'I-LOC',
 10: 'I-EVE'}

In [ ]:
LABEL2IDX

{'O': 0,
 'B-DAT': 1,
 'B-PER': 2,
 'B-ORG': 3,
 'B-LOC': 4,
 'B-EVE': 5,
 'I-DAT': 6,
 'I-PER': 7,
 'I-ORG': 8,
 'I-LOC': 9,
 'I-EVE': 10}

In [21]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=898f014389f7b50b565f9d733f0d9870e020737f6b3bfd0506f0e4e274daad95
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [22]:
label_list = [k for k,v in LABEL2IDX.items()]
label_list

['O',
 'B-DAT',
 'B-PER',
 'B-ORG',
 'B-LOC',
 'B-EVE',
 'I-DAT',
 'I-PER',
 'I-ORG',
 'I-LOC',
 'I-EVE']

In [23]:
import numpy as np
from seqeval.metrics import classification_report
import evaluate

accuracy = evaluate.load('accuracy')
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #############
    cr = classification_report(true_labels, true_predictions,digits=4)
    fname = "report.txt"
    file = open(fname,'w')
    file.write(cr)
    file.close()
    #############
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [25]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig

model_name = '/content/drive/MyDrive/Colab Notebooks/NER-finetuned/finetuned-XLM-R'

config = AutoConfig.from_pretrained(model_name)
# Modify the dropout rates
config.attention_probs_dropout_prob = 0.2  #  default (0.1)
config.hidden_dropout_prob = 0.2  # default (0.1)

model = AutoModelForTokenClassification.from_pretrained(model_name, config=config)

In [26]:
samples = train_dataset[:8]
batch = data_collator([train_dataset[i] for i in range(i)])
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([1, 48]),
 'attention_mask': torch.Size([1, 48]),
 'labels': torch.Size([1, 48])}

In [27]:
 model.config.max_position_embeddings

514

In [28]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 24000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
})


In [29]:
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir = "/tmp/Persina-NER",
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size= TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= EVAL_BATCH_SIZE,
    num_train_epochs= 10,
    weight_decay= 0.05,
    warmup_ratio = 0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    lr_scheduler_type="linear",
    greater_is_better=True,
    push_to_hub=True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.211800,0.171829,0.629725,0.677554,0.652764,0.939501
2,0.208800,0.173825,0.611486,0.711909,0.657887,0.936518
3,0.175200,0.181385,0.602473,0.668002,0.633548,0.933140
4,0.156300,0.182729,0.632285,0.718687,0.672723,0.939527
5,0.137800,0.177154,0.651033,0.699276,0.674293,0.942406
6,0.119300,0.196078,0.641084,0.696195,0.667504,0.938502
7,0.102400,0.205101,0.644292,0.721615,0.680764,0.940684
8,0.090400,0.212393,0.646934,0.702049,0.673365,0.940645


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.211800,0.171829,0.629725,0.677554,0.652764,0.939501
2,0.208800,0.173825,0.611486,0.711909,0.657887,0.936518
3,0.175200,0.181385,0.602473,0.668002,0.633548,0.933140
4,0.156300,0.182729,0.632285,0.718687,0.672723,0.939527
5,0.137800,0.177154,0.651033,0.699276,0.674293,0.942406
6,0.119300,0.196078,0.641084,0.696195,0.667504,0.938502
7,0.102400,0.205101,0.644292,0.721615,0.680764,0.940684
8,0.090400,0.212393,0.646934,0.702049,0.673365,0.940645
9,0.073800,0.229932,0.643651,0.713758,0.676894,0.940395
10,0.067900,0.240838,0.646318,0.719304,0.680860,0.940868


TrainOutput(global_step=30000, training_loss=0.13345700569152832, metrics={'train_runtime': 6839.6216, 'train_samples_per_second': 35.09, 'train_steps_per_second': 4.386, 'total_flos': 9064219682304048.0, 'train_loss': 0.13345700569152832, 'epoch': 10.0})

In [30]:
model.config.num_labels

11

In [31]:
validation_results = trainer.evaluate()

In [32]:
validation_results

{'eval_loss': 0.2408382147550583,
 'eval_precision': 0.6463178294573644,
 'eval_recall': 0.7193036512093668,
 'eval_f1': 0.6808603718556324,
 'eval_accuracy': 0.9408679880132486,
 'eval_runtime': 23.8101,
 'eval_samples_per_second': 125.997,
 'eval_steps_per_second': 15.75,
 'epoch': 10.0}

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
        )
test_results = trainer.evaluate()

In [34]:
test_results

{'eval_loss': 0.25271090865135193,
 'eval_precision': 0.6209295853047593,
 'eval_recall': 0.7027878406048197,
 'eval_f1': 0.65932766900628,
 'eval_accuracy': 0.9380101478049857,
 'eval_runtime': 26.0007,
 'eval_samples_per_second': 115.382,
 'eval_steps_per_second': 14.423}

In [35]:
with open('report.txt') as f:
    for line in f.readlines():
        print(line)

              precision    recall  f1-score   support



         DAT     0.7482    0.8050    0.7756      2256

         EVE     0.5038    0.6055    0.5500       218

         LOC     0.5496    0.6511    0.5961      1863

         ORG     0.5302    0.6119    0.5681       876

         PER     0.5981    0.6734    0.6335      1136



   micro avg     0.6209    0.7028    0.6593      6349

   macro avg     0.5860    0.6694    0.6247      6349

weighted avg     0.6246    0.7028    0.6611      6349

